# sqlparseを触ってみた
* sqlparseによるSQLのフォーマット : https://qiita.com/atsaki/items/eb36a6a7935d525a43e7

In [1]:
import sqlparse
sqlparse.__version__

'0.3.0'

# 練習

In [2]:
sql = '''
select 
  user
  ,country 
from hoge.table as a
left join fuga.table as b
on a.id = b.id 
where type = "HOGE"
'''

In [3]:
print(sql)


select 
  user
  ,country 
from hoge.table as a
left join fuga.table as b
on a.id = b.id 
where type = "HOGE"



In [4]:
print(sqlparse.format(sql, reindent=True, keyword_case='lower'))


select user ,
       country
from hoge.table as a
left join fuga.table as b on a.id = b.id
where type = "HOGE"


In [5]:
print(sqlparse.format(sql, reindent=True, keyword_case='upper'))


SELECT USER ,
       country
FROM hoge.table AS a
LEFT JOIN fuga.table AS b ON a.id = b.id
WHERE TYPE = "HOGE"


In [7]:
# docになかったけどできた
print(sqlparse.format(sql, reindent=True, keyword_case='upper', comma_first = True))


SELECT USER
     , country
FROM hoge.table AS a
LEFT JOIN fuga.table AS b ON a.id = b.id
WHERE TYPE = "HOGE"


In [15]:
# docになかったけどできた
print(sqlparse.format(sql, reindent=True, keyword_case='lower', comma_first = True, indent_after_first = True))


  select user
       , country
  from hoge.table as a
  left join fuga.table as b on a.id = b.id
  where type = "HOGE"


In [6]:
parsed = sqlparse.parse(sql)
stmt = parsed[0]
for t in stmt.tokens:
    print(type(t), t)

<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Token'> select
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.IdentifierList'> user
  ,country
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Token'> from
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Identifier'> hoge.table as a
<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Token'> left join
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Identifier'> fuga.table as b
<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Token'> on
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Comparison'> a.id = b.id
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Where'> where type = "HOGE"



# インデントを4にする

In [29]:
sql = '''
# comment out
select 
  user,
  country 
from hoge.table as a
left join (
   select 
       id,
       type
   from fuga.table
) as b
on a.id = b.id 
where type = "HOGE"
'''

In [30]:
print(
    sqlparse.format(
        sql, 
        reindent = True, 
        reindent_aligned = False,
        indent_tabs = True, 
        indent_width = 1, 
        identifier_case = 'lower', 
        keyword_case='lower',
        strip_comments = False,
        comma_first = True,
        indent_after_first = False
    )
)

# comment out

select user
,  country
from hoge.table as a
left join
	(select id
	,  type
		from fuga.table) as b on a.id = b.id
where type = "HOGE"


# カスタマイズファンクションを作るための構造

In [61]:
from sqlparse import engine
from sqlparse import filters
stack = engine.FilterStack()
stack.postprocess.append(filters.SerializerUnicode())
print(stack.run(sql).__next__())


# comment out
select
  user
  ,country
from hoge.table as a
left join (
   select
       id
   from fuga.table
) as b
on a.id = b.id
where type = "HOGE"



# カンマを先頭に持ってくる
実装するしかないみたい。Githubのコード読むか…

In [79]:
from sqlparse import engine
from sqlparse.filters import ReindentFilter
from sqlparse.sql import Function                                                  

class MyReindentFilter(ReindentFilter):
    def _process_identifierlist(self, tlist):
        identifiers = list(tlist.get_identifiers())
        print(identifiers)
        if len(identifiers) > 1 and not tlist.within(Function):                    

            first = identifiers[0]                                                 
            self.indent += 1                                                       
            tlist.insert_before(first, self.nl())                                  
            self.offset -= 1                                                       
            tlist.insert_after(first, self.nl())      
            
            print(tlist)

            for token in identifiers[1:len(identifiers)-1]:                        
                prev = tlist.token_prev(tlist.token_index(token), False)           
                if prev and prev.is_whitespace():                                  
                    prev.value = ''                                                
                tlist.insert_after(token, self.nl())                               

            last = identifiers[-1]                                                 
            prev = tlist.token_prev(tlist.token_index(last), False)                
            if prev and prev.is_whitespace():                                      
                prev.value = ''                                                    
            self.offset += 1                                                       
            self.indent -= 1                                                       

        self._process_default(tlist) 

In [80]:
stack = engine.FilterStack()
stack.enable_grouping()
stack.stmtprocess.append(MyReindentFilter())
stack.postprocess.append(filters.SerializerUnicode())
stack.run(sql).__next__()

[<Keyword 'user' at 0x1098B3348>, <Identifier 'country' at 0x10989C5E8>]

  user
  
 ,country


AttributeError: 'tuple' object has no attribute 'is_whitespace'

# テーブルネームを取る
うまくいかないな

In [68]:
import sqlparse
from sqlparse.sql import IdentifierList, Identifier
from sqlparse.tokens import Keyword, DML


def is_subselect(parsed):
    if not parsed.is_group:
        return False
    for item in parsed.tokens:
        if item.ttype is DML and item.value.upper() == 'SELECT':
            return True
    return False


def extract_from_part(parsed):
    from_seen = False
    for item in parsed.tokens:
        if from_seen:
            if is_subselect(item):
                for x in extract_from_part(item):
                    yield x
            elif item.ttype is Keyword:
                return
            else:
                yield item
        elif item.ttype is Keyword and item.value.upper() == 'FROM':
            from_seen = True


def extract_table_identifiers(token_stream):
    for item in token_stream:
        if isinstance(item, IdentifierList):
            for identifier in item.get_identifiers():
                yield identifier.get_name()
        elif isinstance(item, Identifier):
            yield item.get_name()
        # It's a bug to check for Keyword here, but in the example
        # above some tables names are identified as keywords...
        elif item.ttype is Keyword:
            yield item.value

def extract_tables(sql):
    stream = extract_from_part(sqlparse.parse(sql)[0])
    return list(extract_table_identifiers(stream))

In [67]:
tables = ', '.join(extract_tables(sql))
print('Tables: {0}'.format(tables))

Tables: a


In [69]:
def extract_tables(sql): 
    parsed = sqlparse.parse(sql)
    stmt = parsed[0]
    for t in stmt.tokens:
        print(type(t), t)
extract_tables(sql)

<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Comment'> # comment out

<class 'sqlparse.sql.Token'> select
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.IdentifierList'> user
  ,country
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Token'> from
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Identifier'> hoge.table as a
<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Token'> left join
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Identifier'> (
   select 
       id
   from fuga.table
) as b
<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Token'> on
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Comparison'> a.id = b.id
<class 'sqlparse.sql.Token'>  
<class 'sqlparse.sql.Token'> 

<class 'sqlparse.sql.Where'> where type = "HOGE"

